# CFP Committee Simulator - Playoff Selection

This notebook applies the official CFP selection protocol to generate the 12-team playoff bracket

In [1]:
# Cell 1: Conference Championship Logic
# Define 2024 FBS conference affiliations
conferences = {
    'SEC': ['Georgia', 'Texas', 'Tennessee', 'Alabama', 'Ole Miss', 'South Carolina', 
            'LSU', 'Missouri', 'Vanderbilt', 'Florida', 'Oklahoma', 'Texas A&M', 
            'Arkansas', 'Auburn', 'Kentucky', 'Mississippi State'],
    'Big Ten': ['Oregon', 'Ohio State', 'Penn State', 'Indiana', 'Michigan', 'Iowa',
                'Illinois', 'Wisconsin', 'Minnesota', 'Nebraska', 'Northwestern', 
                'Michigan State', 'Rutgers', 'UCLA', 'USC', 'Maryland', 'Purdue', 
                'Washington'],
    'ACC': ['SMU', 'Clemson', 'Miami', 'Louisville', 'Duke', 'Syracuse', 'Georgia Tech',
            'Boston College', 'Virginia Tech', 'NC State', 'North Carolina', 'Pittsburgh',
            'Wake Forest', 'California', 'Virginia', 'Florida State', 'Stanford'],
    'Big 12': ['Arizona State', 'Iowa State', 'BYU', 'Colorado', 'Baylor', 'TCU',
               'Kansas State', 'Texas Tech', 'West Virginia', 'Kansas', 'Cincinnati',
               'Oklahoma State', 'UCF', 'Houston', 'Arizona', 'Utah'],
    'Mountain West': ['Boise State', 'UNLV', 'Colorado State', 'San Jose State', 
                      'Fresno State', 'Hawaii', 'Wyoming', 'Nevada', 'Air Force',
                      'New Mexico', 'San Diego State', 'Utah State'],
    'AAC': ['Tulane', 'Army', 'Memphis', 'Navy', 'South Florida', 'UTSA', 'East Carolina',
            'Florida Atlantic', 'North Texas', 'Rice', 'Charlotte', 'Temple',
            'UAB', 'Tulsa'],
    'Sun Belt': ['Louisiana', 'Marshall', 'Georgia State', 'James Madison', 
                 'Arkansas State', 'Texas State', 'South Alabama', 'Troy',
                 'Appalachian State', 'Coastal Carolina', 'Old Dominion', 'Georgia Southern',
                 'Southern Mississippi', 'UL Monroe'],
    'MAC': ['Miami (OH)', 'Ohio', 'Bowling Green', 'Buffalo', 'Toledo', 
            'Western Michigan', 'Northern Illinois', 'Central Michigan',
            'Eastern Michigan', 'Ball State', 'Akron', 'Kent State'],
    'Conference USA': ['Jacksonville State', 'Western Kentucky', 'Liberty', 
                       'Sam Houston State', 'Middle Tennessee', 'New Mexico State',
                       'Louisiana Tech', 'UTEP', 'Florida International', 'Kennesaw State'],
    'Independent': ['Notre Dame', 'UConn', 'Massachusetts']
}

# Create reverse mapping
team_to_conference = {}
for conf, teams in conferences.items():
    for team in teams:
        team_to_conference[team] = conf

def identify_conference_champions(rankings_df):
    """Identify highest ranked team from each conference"""
    champions = {}
    
    for conf in conferences.keys():
        conf_teams = rankings_df[rankings_df['team'].isin(conferences[conf])]
        if not conf_teams.empty:
            # Sort by rank to ensure we get the highest ranked team
            champion = conf_teams.sort_values('rank').iloc[0]
            champions[conf] = {
                'team': champion['team'],
                'rank': champion['rank'],
                'composite_score': champion['composite_score']
            }
    
    return champions

# Load final rankings
import pandas as pd
final_rankings = pd.read_csv('final_rankings.csv')

conference_champs = identify_conference_champions(final_rankings)
print('Conference Champions:')
for conf, champ in conference_champs.items():
    print(f"  {conf:15s}: #{champ['rank']:2d} {champ['team']}")

Conference Champions:
  SEC            : # 6 Georgia
  Big Ten        : # 1 Ohio State
  ACC            : #12 Miami
  Big 12         : #14 Utah
  Mountain West  : #33 Boise State
  AAC            : # 8 South Florida
  Sun Belt       : # 5 James Madison
  MAC            : #25 Toledo
  Conference USA : #53 Kennesaw State
  Independent    : # 2 Notre Dame


In [2]:
# Cell 2: Apply Playoff Selection Rules
def select_playoff_field(rankings_df, conference_champs):
    """
    Apply official CFP selection protocol:
    - 5 highest ranked conference champions
    - 7 at-large bids
    """
    
    # Get 5 highest ranked conference champions
    champ_list = sorted(conference_champs.values(), key=lambda x: x['rank'])
    auto_bid_champs = champ_list[:5]
    auto_bid_teams = [c['team'] for c in auto_bid_champs]
    
    print('Automatic Bids (5 highest conference champions):')
    for champ in auto_bid_champs:
        conf = [k for k, v in conference_champs.items() if v['team'] == champ['team']][0]
        print(f"  #{champ['rank']:2d} {champ['team']:20s} ({conf})")
    
    # Get 7 at-large bids (highest ranked non-auto-bid teams)
    at_large_candidates = rankings_df[~rankings_df['team'].isin(auto_bid_teams)]
    # Sort by rank to ensure we get the 7 highest ranked teams
    at_large_bids = at_large_candidates.sort_values('rank').head(7)
    
    print('\nAt-Large Bids (next 7 highest ranked):')
    for _, team in at_large_bids.iterrows():
        print(f"  #{team['rank']:2d} {team['team']:20s}")
    
    # Combine for playoff field
    playoff_teams = pd.concat([
        rankings_df[rankings_df['team'].isin(auto_bid_teams)],
        at_large_bids
    ]).sort_values('rank')
    
    # Check for conference champion outside top 12
    lowest_champ = auto_bid_champs[-1]
    if lowest_champ['rank'] > 12:
        print(f"\n⚠️  Conference champion #{lowest_champ['rank']} {lowest_champ['team']} displaces at-large team")
    
    return playoff_teams.head(12)

playoff_field = select_playoff_field(final_rankings, conference_champs)

Automatic Bids (5 highest conference champions):
  # 1 Ohio State           (Big Ten)
  # 2 Notre Dame           (Independent)
  # 5 James Madison        (Sun Belt)
  # 6 Georgia              (SEC)
  # 8 South Florida        (AAC)

At-Large Bids (next 7 highest ranked):
  # 3 Indiana             
  # 4 Oregon              
  # 7 Alabama             
  # 9 North Texas         
  #10 Iowa                
  #11 USC                 
  #12 Miami               


In [3]:
# Cell 3: Seed the Bracket
def create_playoff_bracket(playoff_field):
    """
    Seed the 12-team bracket according to CFP rules:
    - Seeds 1-4 get byes
    - Seeds 5-8 host first round
    - Seeds 9-12 visit in first round
    """
    
    # Re-seed 1-12
    playoff_field = playoff_field.reset_index(drop=True)
    playoff_field['seed'] = range(1, 13)
    
    bracket = {
        'first_round': [],
        'quarterfinals': [],
        'byes': []
    }
    
    # Seeds 1-4 get byes to quarterfinals
    for i in range(4):
        team = playoff_field.iloc[i]
        bracket['byes'].append({
            'seed': team['seed'],
            'team': team['team'],
            'rank': team['rank']
        })
    
    # First round matchups (home games for seeds 5-8)
    matchups = [
        (5, 12),  # 5 seed hosts 12 seed
        (6, 11),  # 6 seed hosts 11 seed
        (7, 10),  # 7 seed hosts 10 seed
        (8, 9)    # 8 seed hosts 9 seed
    ]
    
    for home_seed, away_seed in matchups:
        home_team = playoff_field[playoff_field['seed'] == home_seed].iloc[0]
        away_team = playoff_field[playoff_field['seed'] == away_seed].iloc[0]
        
        bracket['first_round'].append({
            'home': {'seed': home_seed, 'team': home_team['team']},
            'away': {'seed': away_seed, 'team': away_team['team']}
        })
    
    return bracket, playoff_field

bracket, seeded_field = create_playoff_bracket(playoff_field)

In [4]:
# Cell 4: Display Bracket
print('=' * 60)
print('12-TEAM COLLEGE FOOTBALL PLAYOFF BRACKET')
print('=' * 60)

print('\n🏆 FIRST ROUND BYES:')
for bye in bracket['byes']:
    print(f"  Seed {bye['seed']}: {bye['team']}")

print('\n🏈 FIRST ROUND GAMES (at higher seed\'s home stadium):')
for i, game in enumerate(bracket['first_round'], 1):
    print(f"  Game {i}: ({game['away']['seed']}) {game['away']['team']:20s} @ "
          f"({game['home']['seed']}) {game['home']['team']}")

print('\n📊 COMPLETE PLAYOFF FIELD:')
print('-' * 50)
print(f"{'Seed':<6} {'Team':<25} {'Overall Rank':<12} {'Record'}")
print('-' * 50)
for _, team in seeded_field.iterrows():
    record = f"{int(team['wins'])}-{int(team['losses'])}"
    print(f"{team['seed']:<6} {team['team']:<25} #{team['rank']:<11} {record}")

12-TEAM COLLEGE FOOTBALL PLAYOFF BRACKET

🏆 FIRST ROUND BYES:
  Seed 1: Ohio State
  Seed 2: Notre Dame
  Seed 3: Indiana
  Seed 4: Oregon

🏈 FIRST ROUND GAMES (at higher seed's home stadium):
  Game 1: (12) Miami                @ (5) James Madison
  Game 2: (11) USC                  @ (6) Georgia
  Game 3: (10) Iowa                 @ (7) Alabama
  Game 4: (9) North Texas          @ (8) South Florida

📊 COMPLETE PLAYOFF FIELD:
--------------------------------------------------
Seed   Team                      Overall Rank Record
--------------------------------------------------
1      Ohio State                #1           9-0
2      Notre Dame                #2           9-0
3      Indiana                   #3           8-0
4      Oregon                    #4           7-1
5      James Madison             #5           9-0
6      Georgia                   #6           8-1
7      Alabama                   #7           7-1
8      South Florida             #8           6-2
9      North T

In [ ]:
# Cell 5: Export Results
import json
from datetime import datetime

# Helper function to convert numpy types to native Python types
def convert_to_native_types(obj):
    """Convert numpy/pandas types to native Python types for JSON serialization."""
    import numpy as np
    if isinstance(obj, (np.int64, np.int32)):
        return int(obj)
    elif isinstance(obj, (np.float64, np.float32)):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    return obj

# Prepare export data (updated to 2025 season)
# Convert DataFrame to dict and handle numpy types
rankings_export = final_rankings[['rank', 'team', 'wins', 'losses', 'composite_score']].head(25).copy()
rankings_export['rank'] = rankings_export['rank'].astype(int)
rankings_export['wins'] = rankings_export['wins'].astype(int)
rankings_export['losses'] = rankings_export['losses'].astype(int)
rankings_export['composite_score'] = rankings_export['composite_score'].astype(float)

playoff_export = seeded_field[['seed', 'team', 'rank', 'wins', 'losses']].copy()
playoff_export['seed'] = playoff_export['seed'].astype(int)
playoff_export['rank'] = playoff_export['rank'].astype(int)
playoff_export['wins'] = playoff_export['wins'].astype(int)
playoff_export['losses'] = playoff_export['losses'].astype(int)

export_data = {
    'timestamp': datetime.now().isoformat(),
    'season': 2025,  # 2025-2026 season
    'week': 15,
    'rankings': rankings_export.to_dict('records'),
    'playoff_field': playoff_export.to_dict('records'),
    'bracket': bracket,
    'weights': {
        'colley_rating': 0.20,
        'massey_rating': 0.25,
        'elo_rating': 0.20,
        'sor': 0.20,
        'win_pct': 0.15
    }
}

# Save to JSON
with open(f'playoff_simulation_2025_week15.json', 'w') as f:
    json.dump(export_data, f, indent=2)

# Save to CSV
final_rankings.to_csv(f'rankings_2025_week15.csv', index=False)
seeded_field.to_csv(f'playoff_field_2025_week15.csv', index=False)

print(f'\n✅ Results exported to:')
print(f'  - playoff_simulation_2025_week15.json')
print(f'  - rankings_2025_week15.csv')
print(f'  - playoff_field_2025_week15.csv')